In [4]:
### Generate n random quintic irreducible polynomials F(x,y) with coefficients having absolute value < ran
### n <=  ran


def gen_poly(ran,n):
    PolyRing2.<x,y> = QQ[]
    PolyRing1.<x> = QQ[]
    const=2*ran
    K1=sample(range(-ran,ran+1),const)
    K2=sample(range(-ran,ran+1),const)
    K3=sample(range(-ran,ran+1),const)
    K4=sample(range(-ran,ran+1),const)
    K5=sample(range(-ran,ran+1),const)
    E=[]
    for i in range(const):
        F=x^5 + K1[i]* x^4 * y + K2[i] * x^3 * y^2 + K3[i]* x^2 * y^3 + K4[i] * x * y^4 + K5[i] * y^5
        F1=PolyRing1(F(x,1))
        if F1.is_irreducible() == True:
            E.append( F )
    if len(E)<n:
        return(E)
    else:
        E=[E[i] for i in range(n)]
        return(E)
    
print(gen_poly(10,5))

[x^5 + 10*x^4*y + 2*x^3*y^2 - 4*x^2*y^3 + x*y^4 + 9*y^5, x^5 + 6*x^4*y + 4*x^3*y^2 + 4*x^2*y^3 + 10*x*y^4 - 10*y^5, x^5 - 4*x^4*y - 8*x^3*y^2 - 6*x^2*y^3 - 7*x*y^4 + 4*y^5, x^5 + 2*x^4*y + 6*x^3*y^2 + x^2*y^3 + 2*x*y^4 + 5*y^5, x^5 - 10*x^4*y + 7*x^3*y^2 - 5*x^2*y^3 - 5*x*y^4 - 4*y^5]


In [5]:
PolyRing2.<x,y> = QQ[]
PolyRing1.<x> = QQ[]

### randomly generated polynomials to be tested

E=[x^5 + 6*x^4*y - 5*x^3*y^2 - 6*x^2*y^3 + 8*x*y^4 - 6*y^5, x^5 - 7*x^4*y - 10*x^3*y^2 - 4*x^2*y^3 + 10*x*y^4 + 5*y^5, x^5 + 8*x^4*y - 2*x^3*y^2 - 8*x^2*y^3 - 5*x*y^4 + 4*y^5, x^5 + 5*x^4*y + 4*x^3*y^2 + 5*x^2*y^3 + 7*x*y^4 - 3*y^5, x^5 - 10*x^4*y + 6*x^3*y^2 + 2*x^2*y^3 + 2*x*y^4 + 9*y^5, x^5 + 3*x^4*y + 7*x^3*y^2 - 3*x^2*y^3 + 4*x*y^4 - 4*y^5, x^5 + 4*x^4*y + 2*x^3*y^2 - 2*x^2*y^3 - 9*x*y^4 + 3*y^5, x^5 - 8*x^4*y + 9*x^3*y^2 + x^2*y^3 + x*y^4 + y^5, x^5 + 7*x^4*y - 8*x^3*y^2 + 8*x^2*y^3 + 3*x*y^4 - y^5, x^5 - 5*x^4*y + 3*x^3*y^2 + 6*x^2*y^3 - 8*x*y^4 + 7*y^5]


In [6]:
### Using modular arithmetic to check the unsolvability in integers of the equation
### Input: F, an irrducible polynomial in Z[x,y];m, an integer; p, n, natural numbers 
###      all prime numbers <= p are used for the reduction
###      all x,y with absolute value <= n are used for the reduction
### Output: 1 means F(x,y)=m is not solvable in integers, 0 means unable to determine or F is reducible

def reduction(F,m,p,n):
    PolyRing2.<x,y> = QQ[]
    PolyRing1.<x> = QQ[]
    poly=F
    div=prime_range(p)
    for i in range(len(div)):
        FF=GF(div[i])   ### finite field
        list=[FF(poly(x,y))-m for x in range(  -n,n+1  )  for y in range(  -n,n+1  ) ]
        if prod(list)!=0:
            return(1)
            break
        else:
            return(0)


In [7]:
### Further sovability check by looking at the norm list
### Input: F(x,y),m
### Output: 1 means maybe solvable, 0 means not solvable or the polynomial x^5-c is reducible



def NormCheck(F,m):
    PolyRing2.<x,y> = QQ[]
    PolyRing1.<x> = QQ[]
    F1=F(x,1)
    poly=PolyRing1(F1)
    if poly.is_irreducible() == True:
        K.<theta> = NumberField(poly)
        norm_list=K.elements_of_norm(m)
        if len(norm_list)>0:
            return(1)
        else:
            return(0)
    else:
        return(0)

In [8]:
### Fucntion of modified height, using the approximation with algdep
### Input : X, a list of n non-integer elements; d, the degree of F(x,y)
### Output: the modified heights of each element of X, and the index of [Q(X):Q]

def HeightMod(X,d):
    n=len(X)
    deg=factorial(d)
    P=[CC(X[i]).algdep(deg) for i in range(n)]
    HM=[max( (P[i]).global_height(),  ln(abs(CC(X[i]))),   1   )   for i in range(n)]
    return(HM)

### Outputs the absolute value of elements as considered in complex numbers
### Prvents "bad operand type" error in some cases

def abs_mod(x):
    return(abs(CC(x)))

In [9]:
### Function of the LLL reduction step
### Input: alpha_0, ALPHA=(alpha_1,...,alpha_n),Z0,p_1,p_2
###      as in the text
### Output: the reduced upper bound


def LLLReduction(alpha0,ALPHA,Z0,p1,p2):           
    from sage.functions.log import logb
    C=10^((1.25*logb(Z0, 10).n(prec=40)).ceiling())
    n=len(ALPHA)
    def TempFunc(C,n,alpha0,ALPHA,Z0,p1,p2):             ### Temporary function
        
        M=matrix.identity(n)
        for i in range(n):
            M[n-2,i]=max( round(C*real_part(  CC(ALPHA[i])  )),1 ) 
            M[n-1,i]=max( round(C*imag_part(  CC(ALPHA[i])  )),1 )
        Y=zero_vector(n)
        Y[n-2]=round(C*real_part(  CC(alpha0)  ))
        Y[n-1]=round(C*imag_part(  CC(alpha0)  ))
        M=M.T
        MLLL=(M.LLL())
        MLLL=MLLL.T
        GS1,GS2=(MLLL).gram_schmidt()
        cc1=max([norm(MLLL.column(0))^2/ norm(GS1.column(i))^2 for i in range(n)])
        sigma=(MLLL).inverse()*Y
        sigma_dist_vec=[min([sigma[i]-floor(sigma[i]),ceil(sigma[i])-sigma[i]]) for i in range(n)]
        sigma_dist_vec=[x for x in sigma_dist_vec if x!= 0]
        sigma_dist=sigma_dist_vec[len(sigma_dist_vec)-1]
        if Y in M.column_space() == False:
            L=cc1^(-1)*sigma_dist*norm(MLLL.column(0))^2
        else:
            L=cc1^(-1)*norm(MLLL.column(0))^2
        return(L)
    
    SS=(n-2)*(Z0)^2   ### S as in the text
    TT=(1+n*Z0)/sqrt(2)   ### T as in the text
    L=TempFunc(C,n,alpha0,ALPHA,Z0,p1,p2)   ### L as in the text
    index=0
    while L^2-SS-TT^2 <0:         ### We need L^2-SS-TT^2 >= 0 
        C=(1.3*C).ceiling()
        L=TempFunc(C,n,alpha0,ALPHA,Z0,p1,p2)
        index=index+1
        if index >15:
            return('Error: LLL-reduction not applicable')
        break

    upper_bound=floor(abs((1/p2)*(ln(C*p1)-ln(sqrt(L^2-SS)-TT) )))
    return(upper_bound)

In [10]:
### Main function, method of T&W
### Input: F(x,y), m
###      F(x,y) must be monic and irreducible
### Output: (abs(x),abs(y)) of all integer solutions pairs (x,y)


def solveTzaWeg(F,m):
    PolyRing2.<x,y> = QQ[]
    PolyRing1.<x> = QQ[]
    poly=PolyRing1(F.subs(y=1))
    deg=poly.degree()
    K.<theta> = NumberField(poly)
    UG = UnitGroup(K)
    TU=[u for u in UG.gens_values() if u.multiplicative_order() < +Infinity]
    FU=[u for u in UG.gens_values() if u.multiplicative_order() == +Infinity]
    rk=len(FU)
    thetas=poly.roots(CC,multiplicities=False)
    norm_list=K.elements_of_norm(m)
    ### Ordering the roots and embeddings so that the real ones are in front, followed by distinct complex ones, then the conjugates 
    
    real_roots=[]
    comp_roots=[]
    for i in thetas:
        if i.is_real() == True:
            real_roots.append(i)
        else:
            comp_roots.append(i)
    r1=len(real_roots)
    r2=(deg-r1)/2
    if r1 < deg:
        ordered_roots=[0 for i in range(deg)]
    for i in range(r1):
        ordered_roots[i]=real_roots[i]
    index=r1
    for i in range(2*r2):
        if comp_roots[i] not in ordered_roots :
            ordered_roots[index]=comp_roots[i]
            ordered_roots[index+r2]=comp_roots[i].conjugate()
            index=index+1
    thetas=ordered_roots
    from sage.rings.number_field.number_field_morphisms import create_embedding_from_approx
    emb=[create_embedding_from_approx(K, thetas[i]) for i in range(deg) ]
    
    
    ### computing c1
    M=matrix(RR,deg,deg)
    for i in range(deg):
        for j in range(deg):
            if j < i:
                M[i,j] = abs(thetas[i]-thetas[j])
            else:
                M[i,j] = abs(thetas[i])+abs(thetas[j])
    c1=2/min(M.list())
    ### Computing c2, c3
    L=[0 for i in range(deg)]
    for i in range(deg):
        L[i]=max([abs((thetas[i]-thetas[j])/(thetas[i]-thetas[k])).n(prec=50) for j in range(deg) for k in range(deg) if j != i if k != j if k != i ])
    c2=max(L)
    c3=c1*c2
    ### Computing c4, c5
    MFU = matrix(RR, rk+1, rk, lambda i, j: ln(abs(emb[i](FU[j]).n(prec=50))))   ### Matrix of log of abs of fundamental units
    V=[(MFU.delete_rows([i])).inverse() for i in range(rk+1)]    ### Inverse of matrix U_I as in the text
    c4=1/(max([V[i].norm(Infinity) for i in range(rk+1)] ))
    c5=c4/(deg-0.9)
    ### Cases A,B 
    ### Computing c6, c7 and the corresponding A1,A2
    M=matrix(RR,deg,len(norm_list))
    for i in range(deg):
        for j in range(len(norm_list)):
            M[i,j]=( abs( emb[i](norm_list[j]) .n(prec=60) ))^(-1)         
    c6=max(M.list())
    c7=min(M.list())
    A1=ln(c6*abs(m))/(c4-(deg-1)*c5)
    A1_min=ln(c6*abs(m))/(c4)
    A2=ln(c7)/(c4-c5)
    ### Case C
    ### Loop all possible abs(theta_i) in the real roots 
    bound_temp=[A1_min,A2,3]
    CC100=ComplexField(100)
    for i in range(deg):
        if thetas[i].is_real() == True:
            JK=[j for j in range(deg) if j != 1]
            j=JK[deg-2]
            k=JK[deg-3]
            alpha2=CC100((emb[i](norm_list[0])/emb[j](norm_list[0])))*CC100((emb[i](FU[0])/emb[j](FU[0])))*((thetas[j]-thetas[k])/(thetas[k]-thetas[i]))
            units=[]
            for ii in range(rk):
                units.append((emb[k](FU[ii])/emb[j](FU[ii])))
            vector=units
            vector.append(alpha2)
            heights=HeightMod( vector ,deg)
            hm0=abs(ln(-1))
            c8=(18*factorial(5)*(4^5)*(256^6)*ln(64)*prod(heights)*hm0)
            A_0=(2/c5)*( ln(2*c3) +c8*ln(3/2) +c8*ln(c8/c5)  ).n(prec=16)
            alpha0=ln(-alpha2)
            ALPHA=units
            ALPHA.append(2*pi*sqrt(-1))
            p1=2*c3
            p2=c5
            bound1=LLLReduction(alpha0,ALPHA,A_0,p1,p2)
            if bound1 >= 50:
                bound2=LLLReduction(alpha0,ALPHA,bound1,p1,p2)
                if bound2 >= 50:
                    bound3=LLLReduction(alpha0,ALPHA,bound2,p1,p2)
                else:
                    bound3=bound2
            else:
                bound3=bound1
            bound_temp.append(bound3)
    bound_final=floor(max(bound_temp))
    
    XY=[]
    from itertools import product
    per=list(product(range(-bound_final,bound_final+1), repeat=rk))   ### Repeated permutation
    for i in range(len(per)):
        result1=prod([FU[j]^(per[i][j]) for j in range(rk)])
        for k in range(len(norm_list)):
            for ii in range(TU[0].multiplicative_order()):
                result2=(norm_list[k])*(TU[0]^ii)*result1
                P=result2.polynomial(var='t')
                if P.degree() <2:
                    a0=abs(result2[0])
                    a1=abs(result2[1])
                    XY.append((a0,a1))
                    
    ### Computing Y0 and loop over all abs(y)<=Y0
    im_roots=[abs(i) for i in thetas if i.is_real()==False]
    c9=max([2^(deg-1)*abs(m)/abs(poly.derivative()(thetas[i])) for i in range(deg)])
    Y0=RR((c9/min(im_roots))^(1/deg))
    Y0=Y0.floor()
    for jj in range(Y0):
        poly2=PolyRing1(F.subs(y=jj))
        int_roots=poly.roots(ZZ,multiplicities=False)
        number1=len(int_roots)
        if number1 > 0:
            for kk in range(number1):
                XY.append((int_roots[kk],jj))

    return(sorted(set(XY)))

In [11]:
### preliminary algorithm


PolyRing2.<x,y> = QQ[]
PolyRing1.<x> = QQ[]
n=5
MM=[i for i in range(1,n+1)]
L1=[]
for i in range(len(E)):
    for j in range(len(MM)):
        const=NormCheck(E[i],MM[j])
        if const==1:
            if reduction(E[i],MM[j],10,5) == 0:
                L1.append((E[i],MM[j]))
                    

print('total number of equations considered =',len(E)*len(MM))
print('out of which passed the insolvability test and hence are up to further inspection =',len(L1))

total number of equations considered = 50
out of which passed the insolvability test and hence are up to further inspection = 34


In [12]:
print(L1)

[(x^5 + 6*x^4*y - 5*x^3*y^2 - 6*x^2*y^3 + 8*x*y^4 - 6*y^5, 1), (x^5 + 6*x^4*y - 5*x^3*y^2 - 6*x^2*y^3 + 8*x*y^4 - 6*y^5, 2), (x^5 + 6*x^4*y - 5*x^3*y^2 - 6*x^2*y^3 + 8*x*y^4 - 6*y^5, 3), (x^5 + 6*x^4*y - 5*x^3*y^2 - 6*x^2*y^3 + 8*x*y^4 - 6*y^5, 4), (x^5 + 6*x^4*y - 5*x^3*y^2 - 6*x^2*y^3 + 8*x*y^4 - 6*y^5, 5), (x^5 - 7*x^4*y - 10*x^3*y^2 - 4*x^2*y^3 + 10*x*y^4 + 5*y^5, 1), (x^5 - 7*x^4*y - 10*x^3*y^2 - 4*x^2*y^3 + 10*x*y^4 + 5*y^5, 4), (x^5 - 7*x^4*y - 10*x^3*y^2 - 4*x^2*y^3 + 10*x*y^4 + 5*y^5, 5), (x^5 + 8*x^4*y - 2*x^3*y^2 - 8*x^2*y^3 - 5*x*y^4 + 4*y^5, 1), (x^5 + 8*x^4*y - 2*x^3*y^2 - 8*x^2*y^3 - 5*x*y^4 + 4*y^5, 2), (x^5 + 8*x^4*y - 2*x^3*y^2 - 8*x^2*y^3 - 5*x*y^4 + 4*y^5, 4), (x^5 + 8*x^4*y - 2*x^3*y^2 - 8*x^2*y^3 - 5*x*y^4 + 4*y^5, 5), (x^5 + 5*x^4*y + 4*x^3*y^2 + 5*x^2*y^3 + 7*x*y^4 - 3*y^5, 1), (x^5 + 5*x^4*y + 4*x^3*y^2 + 5*x^2*y^3 + 7*x*y^4 - 3*y^5, 3), (x^5 + 5*x^4*y + 4*x^3*y^2 + 5*x^2*y^3 + 7*x*y^4 - 3*y^5, 5), (x^5 - 10*x^4*y + 6*x^3*y^2 + 2*x^2*y^3 + 2*x*y^4 + 9*y^5, 1), 

In [11]:
### solve the equations in L1 using the method of T&W
### took about 35 min in total on my laptop
### separate in two parts, as now we only have 45 min of free trial

L1_sol=[]
for i in range(16):                 ### the first half, takes about 15 min
    sol=solveTzaWeg(L1[i][0],L1[i][1])
    L1_sol.append((L1[i],sol))
    print((L1[i],sol))

((x^5 + 6*x^4*y - 5*x^3*y^2 - 6*x^2*y^3 + 8*x*y^4 - 6*y^5, 1), [(1, 0)])


((x^5 + 6*x^4*y - 5*x^3*y^2 - 6*x^2*y^3 + 8*x*y^4 - 6*y^5, 2), [(1, 1)])


((x^5 + 6*x^4*y - 5*x^3*y^2 - 6*x^2*y^3 + 8*x*y^4 - 6*y^5, 3), [])


((x^5 + 6*x^4*y - 5*x^3*y^2 - 6*x^2*y^3 + 8*x*y^4 - 6*y^5, 4), [])


((x^5 + 6*x^4*y - 5*x^3*y^2 - 6*x^2*y^3 + 8*x*y^4 - 6*y^5, 5), [])


...

((x^5 + 8*x^4*y - 2*x^3*y^2 - 8*x^2*y^3 - 5*x*y^4 + 4*y^5, 5), [])


((x^5 + 5*x^4*y + 4*x^3*y^2 + 5*x^2*y^3 + 7*x*y^4 - 3*y^5, 1), [(1, 0)])


((x^5 + 5*x^4*y + 4*x^3*y^2 + 5*x^2*y^3 + 7*x*y^4 - 3*y^5, 3), [(0, 1)])


((x^5 + 5*x^4*y + 4*x^3*y^2 + 5*x^2*y^3 + 7*x*y^4 - 3*y^5, 5), [(1, 1)])


((x^5 - 10*x^4*y + 6*x^3*y^2 + 2*x^2*y^3 + 2*x*y^4 + 9*y^5, 1), [(1, 0)])


In [13]:
L1_sol=[]
for i in range(16,len(L1)):                 ### the second half, takes about 20 min
    sol=solveTzaWeg(L1[i][0],L1[i][1])
    L1_sol.append((L1[i],sol))
    print((L1[i],sol))

((x^5 - 10*x^4*y + 6*x^3*y^2 + 2*x^2*y^3 + 2*x*y^4 + 9*y^5, 2), [])


((x^5 - 10*x^4*y + 6*x^3*y^2 + 2*x^2*y^3 + 2*x*y^4 + 9*y^5, 4), [])


((x^5 - 10*x^4*y + 6*x^3*y^2 + 2*x^2*y^3 + 2*x*y^4 + 9*y^5, 5), [])


((x^5 + 3*x^4*y + 7*x^3*y^2 - 3*x^2*y^3 + 4*x*y^4 - 4*y^5, 1), [(1, 0)])


((x^5 + 3*x^4*y + 7*x^3*y^2 - 3*x^2*y^3 + 4*x*y^4 - 4*y^5, 2), [])


...

((x^5 + 7*x^4*y - 8*x^3*y^2 + 8*x^2*y^3 + 3*x*y^4 - y^5, 4), [])


((x^5 + 7*x^4*y - 8*x^3*y^2 + 8*x^2*y^3 + 3*x*y^4 - y^5, 5), [])


((x^5 - 5*x^4*y + 3*x^3*y^2 + 6*x^2*y^3 - 8*x*y^4 + 7*y^5, 1), [(1, 0)])


((x^5 - 5*x^4*y + 3*x^3*y^2 + 6*x^2*y^3 - 8*x*y^4 + 7*y^5, 3), [])


((x^5 - 5*x^4*y + 3*x^3*y^2 + 6*x^2*y^3 - 8*x*y^4 + 7*y^5, 4), [(1, 1)])


In [0]:
###
###
### 

In [0]:
### Method of Bilu and Hanrot
###
###

In [14]:
### Main function, method of B&H
### Input :F(x,y), m
###      F(x,y) must be monic and irreducible
### Output: absolute values of the solution pairs



def solveBiluHanrot(F,m):
    CC500=ComplexField(500)
    ComplexNumber=CC500
    PolyRing2.<x,y> = QQ[]
    PolyRing1.<x> = QQ[]
    poly=PolyRing1(F.subs(y=1))
    deg=poly.degree()
    K.<theta> = NumberField(poly)
    UG = UnitGroup(K)
    TU=[u for u in UG.gens_values() if u.multiplicative_order() < +Infinity]
    FU=[u for u in UG.gens_values() if u.multiplicative_order() == +Infinity]
    rk=len(FU)
    thetas=poly.roots(CC,multiplicities=False)
    norm_list=K.elements_of_norm(m)
    ### Ordering the roots and embeddings so that the real ones are in front, followed by distinct complex ones, then the conjugates 
    
    real_roots=[]
    comp_roots=[]
    for i in thetas:
        if i.is_real() == True:
            real_roots.append(i)
        else:
            comp_roots.append(i)
    r1=len(real_roots)
    r2=(deg-r1)/2
    if r1 < deg:
        ordered_roots=[0 for i in range(deg)]
    for i in range(r1):
        ordered_roots[i]=real_roots[i]
    index=r1
    for i in range(2*r2):
        if comp_roots[i] not in ordered_roots :
            ordered_roots[index]=comp_roots[i]
            ordered_roots[index+r2]=comp_roots[i].conjugate()
            index=index+1
    thetas=ordered_roots
    from sage.rings.number_field.number_field_morphisms import create_embedding_from_approx
    emb=[create_embedding_from_approx(K, thetas[i]) for i in range(deg) ]
    
    
    ### computing c1
    M=matrix(RR,deg,deg)
    for i in range(deg):
        for j in range(deg):
            if j < i:
                M[i,j] = abs(thetas[i]-thetas[j])
            else:
                M[i,j] = abs(thetas[i])+abs(thetas[j])
    c1=2/min(M.list())
    ### Computing c2, c3
    ### Might take long time, as there are deg^3 computations
    L=[0 for i in range(deg)]
    for i in range(deg):
        L[i]=max([abs((thetas[i]-thetas[j])/(thetas[i]-thetas[k])) for j in range(deg) for k in range(deg) if j != i if k != j if k != i ])
    c2=max(L)
    c3=c1*c2
    ### Computing c4, c5
    MFU = matrix(RR, rk+1, rk, lambda i, j: ln(abs(emb[i](FU[j]).n(prec=50))))   ### Matrix of log of abs of fundamental units
    V=[(MFU.delete_rows([i])).inverse() for i in range(rk+1)]    ### Inverse of matrix U_I as in the text
    c4=1/(max([V[i].norm(Infinity) for i in range(rk+1)] ))
    c5=c4/(deg-0.9)
    ### Cases A,B 
    ### Computing c6, c7 and the corresponding A1,A2
    M=matrix(RR,deg,len(norm_list))
    for i in range(deg):
        for j in range(len(norm_list)):
            M[i,j]=( abs( CC(emb[i](norm_list[j]))  ))^(-1)         
    c6=max(M.list())
    c7=min(M.list())
    A1=ln(c6*abs(m))/(c4-(deg-1)*c5)
    A1_min=ln(c6*abs(m))/(c4)
    A2=ln(c7)/(c4-c5)
    ### Case C
    ### Computing c8 and an initial bound on A
    
    
    bound_temp=[A1_min,A2,3]
    CC100=ComplexField(100)
    for i in range(deg):
        if thetas[i].is_real() == True:
            JK=[j for j in range(deg) if j != 1]
            j=JK[deg-2]
            k=JK[deg-3]
            alpha2=CC100((emb[i](norm_list[0])/emb[j](norm_list[0])))*CC100((emb[i](FU[0])/emb[j](FU[0])))*((thetas[j]-thetas[k])/(thetas[k]-thetas[i]))
            units=[]
            for ii in range(rk):
                units.append((emb[k](FU[ii])/emb[j](FU[ii])))
            vector=units
            vector.append(alpha2)
            heights=HeightMod( vector ,deg)
            hm0=abs(ln(-1))
            c8=(18*factorial(5)*(4^5)*(256^6)*ln(64)*prod(heights)*hm0)
            A_0=RR((2/c5)*( ln(2*c3) +c8*ln(3/2) +c8*ln(c8/c5)  )).ceil()
    
    
            MFU = matrix( CC500, rk+1, rk, lambda i, j: ln( abs(CC500(emb[i](FU[j]))) ) )  
            II=[]
            for i in range(rk+1):
                II.append(  [i,[j for j in range(rk+1) if j!=i]]   )
            V=[(MFU.delete_rows([i])).inverse() for i in range(rk+1)]    ### V_I, inverse of matrix U_I as in the text
            K.<theta> = NumberField(poly) ### Redefine theta as in the number field, otherwise it's recognised as in the symbolic ring
            import numpy
            delta=[sum(numpy.abs(V[i].row(j).list())) for i in range(rk+1) for j in range(rk)]
            u=norm_list[0]
            LL=[[ln( abs_mod(  ((emb[II[i][0]](theta)- emb[j](theta))/emb[j](u)) )   ) for j in II[i][1] ] for i in range(rk+1)]
            LV=[(MFU.delete_rows([i])).inverse() for i in range(rk+1)]
            for i in range(rk):
                for j in range(rk+1):
                    LV[j].set_col_to_multiple_of_col(i, i, LL[j][i])
            ### Calculating c9, c10, c11, lamdba_k's
            lam=[sum(LV[i].row(j)) for i in range(rk+1) for j in range(rk)]
            c10=max(abs(delta[0]),abs(delta[1]))
            c11=max(((abs(V[0][0,0])+abs(V[0][0,1]))/deg )+abs(lam[0]),  ((abs(V[0][1,0])+abs(V[0][1,1]))/deg )+abs(lam[1]))
            c9=max([2^(deg-1)*abs(m)/abs(poly.derivative()(thetas[i])) for i in range(deg)])
            M=matrix(CC,deg,deg)
            for i in range(deg):
                for j in range(deg):
                    if j < i:
                        M[i,j] = abs(thetas[i]-thetas[j])
                    else:
                        M[i,j] = abs(thetas[i])+abs(thetas[j])
            c1=2/min(M.list())
            c14=c1*c9*e^(deg*c11/c10)
            c15=deg/c10
            c16=c14*(V[0].norm(Infinity))
            ### Calculating S_g, T_g
            delta_h=max(abs(delta[0]),abs(delta[1]))
            h=0
            if delta_h in delta == True:
                h=delta.index(delta_h)
            else:
                if -delta_h in delta == True:
                    h=delta.index(-delta_h)
            if h == 0 :
                g=1
            else:
                g=0
            S_g=delta[g]/delta[h]
            T_g=(delta[g]*lam[h]-delta[h]*lam[g])/delta[h]
            k=50
            ### convergent of S_g with the smallest denominator > X0
            ### Funtion to find q as in the text, using continued fractions
            def cfreduction(S_g,A_0,k):
                X0=k*A_0
                index=0
                c=continued_fraction(S_g)
                while abs(c.numerator(index)-(S_g)*(c.denominator(index))) > 1/X0:
                    index=index+1
                    if c.denominator(index) > X0:
                        break
                q=c.denominator(index)
                return(q,index)
            k=50
            q=cfreduction(S_g,A_0,k)[0]
            index=cfreduction(S_g,A_0,k)[1]
            RR500=RealField(500)
            dist=min(  RR500(q*(T_g)).ceil()-q*(T_g),  q*(T_g)-RR500(q*(T_g)).floor()  )   ### distance to neareast integer of q*T_g
            while dist < 2/k:
                k=2*k
                q=cfreduction(S_g,A_0,k)
                if k > 10000:
                    break
            new_bound=RR((1/c15)*ln(2*c16*k^2*A_0)).ceil() 
            bound_temp.append(new_bound)
    bound_final=floor(max(bound_temp))
    
    XY=[]
    from itertools import product
    per=list(product(range(-bound_final,bound_final+1), repeat=rk))   ### Repeated permutation
    for i in range(len(per)):
        result1=prod([FU[j]^(per[i][j]) for j in range(rk)])
        for k in range(len(norm_list)):
            for ii in range(TU[0].multiplicative_order()):
                result2=(norm_list[k])*(TU[0]^ii)*result1
                P=result2.polynomial(var='t')
                if P.degree() <2:
                    a0=abs(result2[0])
                    a1=abs(result2[1])
                    XY.append((a0,a1))
    
    ### Computing Y2 and loop over all abs(y)<=Y2
    min_roots_diff=min([abs(thetas[i]-thetas[j]) for i in range(deg) for j in range(deg) if j!=i])
    Y2=RR((((deg+1)*c9)/min_roots_diff)^(1/deg))
    Y2=Y2.floor()
    for jj in range(-Y2,Y2+1):
        poly2=PolyRing1(F.subs(y=jj))
        int_roots=poly.roots(ZZ,multiplicities=False)
        number1=len(int_roots)
        if number1 > 0:
            for kk in range(number1):
                XY.append((int_roots[kk],jj))
    return(sorted(set(XY)))

In [15]:
### solve the equations in L1 using the method of B&H
### took about 5 min on my laptop, faster than the method of T&W

BH_sol=[]
for i in range(len(L1)):
    sol=solveBiluHanrot(L1[i][0],L1[i][1])
    BH_sol.append((L1[i],sol))
    print((L1[i],sol))

((x^5 + 6*x^4*y - 5*x^3*y^2 - 6*x^2*y^3 + 8*x*y^4 - 6*y^5, 1), [(1, 0)])


((x^5 + 6*x^4*y - 5*x^3*y^2 - 6*x^2*y^3 + 8*x*y^4 - 6*y^5, 2), [(1, 1)])


((x^5 + 6*x^4*y - 5*x^3*y^2 - 6*x^2*y^3 + 8*x*y^4 - 6*y^5, 3), [])


((x^5 + 6*x^4*y - 5*x^3*y^2 - 6*x^2*y^3 + 8*x*y^4 - 6*y^5, 4), [])


((x^5 + 6*x^4*y - 5*x^3*y^2 - 6*x^2*y^3 + 8*x*y^4 - 6*y^5, 5), [])


...

((x^5 + 7*x^4*y - 8*x^3*y^2 + 8*x^2*y^3 + 3*x*y^4 - y^5, 4), [])


((x^5 + 7*x^4*y - 8*x^3*y^2 + 8*x^2*y^3 + 3*x*y^4 - y^5, 5), [])


((x^5 - 5*x^4*y + 3*x^3*y^2 + 6*x^2*y^3 - 8*x*y^4 + 7*y^5, 1), [(1, 0)])


((x^5 - 5*x^4*y + 3*x^3*y^2 + 6*x^2*y^3 - 8*x*y^4 + 7*y^5, 3), [])


((x^5 - 5*x^4*y + 3*x^3*y^2 + 6*x^2*y^3 - 8*x*y^4 + 7*y^5, 4), [(1, 1)])
